<font size="10">Autoencoder Training for Action-Angle Coordinates of the Kepler Problem</font>

<font size="4">This script trains an autoencoder neural network to discover an action-angle coordinate transformation for the Kepler problem. The Kepler problem is a second order Hamiltonian ODE, given by</font>where $r = r(t) \geq 0$ denotes the distance between to bodies interacting through a central force that varies as the inverse square of the distance between them (Newtonian gravity).

$\ddot{r} = \frac{1}{r^3} - \frac{1}{r^2},$

<font size="4">where $r = r(t) \geq 0$ denotes the distance between to bodies interacting through a central force that varies as the inverse square of the distance between them (Newtonian gravity). This notebook generates the training data from the Kepler problem and trains an autoencoder neural network. The training is done through random initializations of multiple network architectures and then saving the best performing network on both the training and validation data. Since multiple networks are trained, this notebook may take more than 1 hour to complete. If you do not have time to wait, there is a saved network and the notebook ActionAngle.ipynb that loads and prints the results to view.</font>

<font size="4">This code was written by [Bethany Lusch](https://bethanyl.github.io/) and accompanies the work in Section 6.3 of the book <i>Data-Driven Methods for Dynamic Systems</i>. We use Tensorflow 2.0 to build the neural network, which is imported as tf, while also using augmented versions of the scripts from the [PDEKoopman2](https://github.com/CraigGin/PDEKoopman2) repo, written by [Craig Gin](https://scholar.google.com/citations?user=w4mzyIAAAAAJ&hl=en).

In [2]:
import random as r
import numpy as np
import sys

# Tensorflow imports
from tensorflow import keras
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.activations import relu

# Add the components for the PDEKoopman2 repo
from action_angle_scripts.utils import run_experiment, getdatasize
from action_angle_scripts.DenseResBlock import DenseResBlock
from action_angle_scripts.DenseBlock import DenseBlock
from action_angle_scripts.RelMSE import RelMSE

In [2]:
# Generate training data

# Imports specific to generating training data
from scipy.integrate import solve_ivp
from numpy.random import default_rng

def func(t, x):
    return [x[1], (x[0]+1)**(-3) - (x[0]+1)**(-2)]

def potential_fn(x):
    return (1/2)*(x[:,1]**2) + (1/2)*(x[:,0]+1)**(-2) - (x[:,0]+1)**(-1)

def create_data(numICs, seed):
    rng = default_rng(seed)
    
    # select a bunch of initial conditions in the correct range
    ic_ranges = np.array([[5.0, 6.0], [0.0, 0.0]])
    ic_spans = ic_ranges[:,1] - ic_ranges[:,0]
    
    initial_conds = rng.random((10*numICs,2))
    initial_conds = initial_conds * ic_spans + ic_ranges[:,0]
    initial_conds[-1 + 1,:] = [0,0]
    
    lenT = 501
    data = np.zeros((numICs, lenT, 2))
    
    t_span = [0, 10]
    t = np.linspace(0, 10, lenT) # tSpan = 0:0.02:1
    for j in np.arange(numICs):
        sol = solve_ivp(func, t_span, initial_conds[j,:], t_eval=t)
        data[j,:,:] = (sol.y).transpose() # shape [2, 51]
    return data

# Save data for reproducibility
data_prefix = 'Kepler_expt'
n_IC_train = 200  # Number of initial conditions
n_IC_val = 100

train_data = create_data(n_IC_train, seed=1)
np.save("{}_train1_x.npy".format(data_prefix) , train_data, allow_pickle=False)

val_data = create_data(n_IC_val, seed=2)
np.save("{}_val_x.npy".format(data_prefix) , val_data, allow_pickle=False)

In [3]:
# Experiment name for saving results
expt_name = 'Kepler_expt'
data_file_prefix = 'Kepler_expt'

# Set size of latent space, and retrieve the size of the data
n_latent = 2
_, len_time, n_inputs = getdatasize(data_file_prefix)

# Set other parameters
data_train_len = 1  # Number of training data files
L_diag = False  # Whether the dynamics matrix is forced to be diagonal
num_shifts = 30  # Number of time steps to include in prediction loss
num_shifts_middle = 30  # Number of time steps to include in linearity loss
loss_weights = [1, 1, 1, 1, 1]  # Weights of 5 loss functions

# Set up encoder and decoder configuration dict(s)
activation = relu
initializer = keras.initializers.VarianceScaling()
regularizer = l1_l2(0, 1e-8)

hidden_config = {'activation': activation,
                 'kernel_initializer': initializer,
                 'kernel_regularizer': regularizer}

output_config = {'activation': None,
                 'kernel_initializer': initializer,
                 'kernel_regularizer': regularizer}

outer_config = {'n_inputs': n_inputs,
                'num_hidden': 4,
                'hidden_width': 30,
                'hidden_config': hidden_config,
                'output_config': output_config}

inner_config = {'kernel_regularizer': regularizer}

# The following parameterizes the linear matrix, denoted L, using the auxiliary network, denoted Omega
L_param = True

# auxiliary_network maps from the encoded state (n_latent) to n_latent/2 for each item in batch
# L is block-diagonal matrix formed of output of auxiliary_network
aux_config = {'n_inputs': 1,
                'num_hidden': 3,
                'hidden_width': 10,
                'output_width': 1,
                'hidden_config': hidden_config,
                'output_config': output_config}

# Set up network configuration dict
network_config = {'n_inputs': n_inputs,
                  'n_latent': n_latent,
                  'len_time': len_time,
                  'num_shifts': num_shifts,
                  'num_shifts_middle': num_shifts_middle,
                  'outer_encoder': DenseResBlock(**outer_config),
                  'outer_decoder': DenseResBlock(**outer_config),
                  'inner_config': inner_config,
                  'L_diag': L_diag, 
                  'L_param': L_param,
                  'auxiliary_network':DenseBlock(**aux_config),
                  'delta_t': 0.02}

# Aggregate all the training options in one dictionary
training_options = {'aec_only_epochs': 10,
                    'init_full_epochs': 30, #15,
                    'best_model_epochs': 500, #500,
                    'num_init_models': 10, #10,
                    'loss_fn': RelMSE(),
                    'optimizer': keras.optimizers.Adam,
                    'optimizer_opts': {},
                    'batch_size': 50,
                    'data_train_len': data_train_len,
                    'loss_weights': loss_weights}

In [4]:
# Train Autoencoder/Conjugacy/Action-Angle network

# Get a random number generator seed
random_seed = r.randint(0, 10**(10))

# Set the custom objects used in the model (for loading purposes)
custom_objs = {"RelMSE": RelMSE}

# Run the experiment - save the best trained model
run_experiment(random_seed=random_seed,
               expt_name=expt_name,
               data_file_prefix=data_file_prefix,
               training_options=training_options,
               network_config=network_config,
               custom_objects=custom_objs)

/Users/jbramburger7/opt/miniconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
4/4 [==============================] - 2s 183ms/step - loss: 0.1295 - output_1_loss: 0.0647 - output_2_loss: 0.0648 - output_3_loss: 0.0000e+00 - val_loss: 0.1214 - val_output_1_loss: 0.0606 - val_output_2_loss: 0.0608 - val_output_3_loss: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 0s 47ms/step - loss: 0.1170 - output_1_loss: 0.0584 - output_2_loss: 0.0586 - output_3_loss: 0.0000e+00 - val_loss: 0.1089 - val_output_1_loss: 0.0543 - val_output_2_loss: 0.0547 - val_output_3_loss: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 0s 47ms/step - loss: 0.1047 - output_1_loss: 0.0521 - output_2_loss: 0.0526 - output_3_loss: 0.0000e+00 - val_loss: 0.0971 - val_output_1_loss: 0.0483 - val_output_2_loss: 0.0488 - val_output_3_loss: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 0s 50ms/step - loss: 0.0934 - output_1_loss: 0.0464 - output_2_loss: 0.0470 - output_3_loss: 0.0000e+00 - val_loss: 0.0865 - val_output_1_loss: 0.0429 - val_output_2_

INFO:tensorflow:Assets written to: ../model_weights/Kepler_expt/model_0/assets


INFO:tensorflow:Assets written to: ../model_weights/Kepler_expt/model_0/assets
/Users/jbramburger7/opt/miniconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10


...

INFO:tensorflow:Assets written to: ../model_weights/Kepler_expt/final_model/assets


INFO:tensorflow:Assets written to: ../model_weights/Kepler_expt/final_model/assets


INFO:tensorflow:Assets written to: ../results/Kepler_expt/final_model/assets


INFO:tensorflow:Assets written to: ../results/Kepler_expt/final_model/assets


Best model saved to: ../model_weights/Kepler_expt/final_model
Exported training dictionaries to:  ../results/Kepler_expt/
